# Similarity Queries

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2023-04-04 11:33:23,167 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-04-04 11:33:23,169 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2023-04-04 11:33:23,170 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2023-04-04T11:33:23.170615', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


# Similarity interface

In [3]:
from gensim import models
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

2023-04-04 11:33:56,946 : INFO : using serial LSI version on this node
2023-04-04 11:33:56,948 : INFO : updating model with new documents
2023-04-04 11:33:56,949 : INFO : preparing a new chunk of documents
2023-04-04 11:33:56,952 : INFO : using 100 extra samples and 2 power iterations
2023-04-04 11:33:56,953 : INFO : 1st phase: constructing (12, 102) action matrix
2023-04-04 11:33:56,958 : INFO : orthonormalizing (12, 102) action matrix
2023-04-04 11:33:56,961 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2023-04-04 11:33:56,963 : INFO : computing the final decomposition
2023-04-04 11:33:56,965 : INFO : keeping 2 factors (discarding 43.156% of energy spectrum)
2023-04-04 11:33:56,980 : INFO : processed documents up to #9
2023-04-04 11:33:56,984 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"time" + 0.265*"response" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2023-04-04 11:33:56,986 : INFO : topic #1(2

In [4]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

[(0, 0.46182100453271535), (1, -0.07002766527900053)]


# Initializing query structures

In [5]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it

2023-04-04 11:34:44,407 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2023-04-04 11:34:44,411 : INFO : creating matrix with 9 documents and 2 features


/!\ Warning : The class similarities.MatrixSimilarity is only appropriate when the whole set of vectors fits into memory. For example, a corpus of one million documents would require 2GB of RAM in a 256-dimensional LSI space, when used with this class.

In [6]:
index.save('./tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('./tmp/deerwester.index')

2023-04-04 11:36:52,329 : INFO : MatrixSimilarity lifecycle event {'fname_or_handle': './tmp/deerwester.index', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-04-04T11:36:52.329216', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'saving'}
2023-04-04 11:36:52,334 : INFO : saved ./tmp/deerwester.index
2023-04-04 11:36:52,337 : INFO : loading MatrixSimilarity object from ./tmp/deerwester.index
2023-04-04 11:36:52,351 : INFO : MatrixSimilarity lifecycle event {'fname': './tmp/deerwester.index', 'datetime': '2023-04-04T11:36:52.351001', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'loaded'}


# Performing queries

In [7]:
sims = index[vec_lsi]  # perform a similarity query against the corpus
print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

[(0, 0.998093), (1, 0.93748635), (2, 0.9984453), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.09879464), (8, 0.050041765)]


In [8]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for doc_position, doc_score in sims:
    print(doc_score, documents[doc_position])

0.9984453 The EPS user interface management system
0.998093 Human machine interface for lab abc computer applications
0.9865886 System and human system engineering testing of EPS
0.93748635 A survey of user opinion of computer system response time
0.90755945 Relation of user perceived response time to error measurement
0.050041765 Graph minors A survey
-0.09879464 Graph minors IV Widths of trees and well quasi ordering
-0.10639259 The intersection graph of paths in trees
-0.12416792 The generation of random binary unordered trees
